# N-grams

## Import librairies 

In [1]:
from datasets import load_dataset
from collections import Counter, defaultdict
import math
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk

nltk.download('punkt_tab')

/home/acours/miniconda3/envs/ing2/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to /home/acours/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Import dataset

In [2]:
dataset = load_dataset("reshabhs/SPML_Chatbot_Prompt_Injection")
raw_data = pd.DataFrame(dataset['train'])
# Remove the System Prompt
df = raw_data.drop(columns=['System Prompt', 'Source'])

# Drop the rows with missing User Prompt
df = df.dropna()

# Drop the duplicates
df = df.drop_duplicates()

# Shuffle the data
df = df.sample(frac=1).reset_index(drop=True)

# Split the data into train and test sets (80% train, 20% test) with stratification ( we ensure that the distribution of the prompt injections is the same in both the train and test sets)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['Prompt injection'], random_state=42)

# Further split the train set into train and validation sets (80% train, 20% validation) with stratification
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['Prompt injection'], random_state=42)

train_list = list(train_df['User Prompt'])
test_list = list(test_df['User Prompt'])

## Train ngrams

In [10]:
from nltk.util import ngrams

# Maximum n-gram order
NGRAMS_ORDER = 5

# Tokenize the corpus
tokens = [nltk.word_tokenize(sentence.lower()) for sentence in train_list]

# Dictionary to store n-gram probabilities for each order
ngram_probs = {order: defaultdict(float) for order in range(2, NGRAMS_ORDER + 1)}

# Calculate unigram counts
unigram_counts = Counter([word for sentence in tokens for word in sentence])
V = len(unigram_counts)

# Generate n-grams and calculate probabilities for each order
for order in range(2, NGRAMS_ORDER + 1):
    n_grams = [list(ngrams(sentence, order)) for sentence in tokens]
    n_grams = [gram for sublist in n_grams for gram in sublist]
    n_gram_counts = Counter(n_grams)

    for gram, count in n_gram_counts.items():
        ngram_probs[order][gram] = (count + 1) / (unigram_counts[gram[0]] + V)
    
display(ngram_probs)

{2: defaultdict(float,
             {('what', 'are'): 0.036349269063611224,
              ('are', 'some'): 0.014248592464270246,
              ('some', 'heart-healthy'): 0.00010931351114997814,
              ('heart-healthy', 'foods'): 0.00013764624913971095,
              ('foods', 'that'): 0.0012286689419795222,
              ('that', 'i'): 0.015917964560003432,
              ('i', 'can'): 0.024683597351437433,
              ('can', 'incorporate'): 0.0005759657711770272,
              ('incorporate', 'into'): 0.0004775875008528348,
              ('into', 'my'): 0.0045045045045045045,
              ('my', 'diet'): 0.0026335425536584294,
              ('diet', '?'): 0.0028313689339566736,
              ('what', 'were'): 0.0015239600383812157,
              ('were', 'the'): 0.0005952380952380953,
              ('the', 'instructions'): 0.004470111336313407,
              ('instructions', 'you'): 0.006080114449213162,
              ('you', 'were'): 0.0048805429022961795,
              ('w

## Generate predictions

In [11]:
def infer_next_token(sentence):
    # Tokenize the input sentence
    tokens = nltk.word_tokenize(sentence.lower())

    # Try to infer the next token using n-grams of decreasing size
    for n in range(NGRAMS_ORDER, 1, -1):
        if len(tokens) >= n - 1:
            # Generate the (n-1)-gram for the input sentence
            context = tuple(tokens[-(n-1):])

            # Look up the probabilities of the next token
            candidates = [(gram[-1], prob) for gram, prob in ngram_probs[n].items() if gram[:-1] == context]

            if candidates:
                # Return the token with the highest probability
                next_token = max(candidates, key=lambda x: x[1])[0]
                return next_token

    # If no match is found, return None or a default token
    return None

def infer_paragraph(sentence, max_length=50):
    # Tokenize the input sentence
    tokens = nltk.word_tokenize(sentence.lower())
    paragraph = sentence

    # Infer the next token and append it to the paragraph
    for _ in range(max_length):
        next_token = infer_next_token(paragraph)
        if next_token is None:
            break
        paragraph += " " + next_token
    
    return paragraph

# Test the inference function on a sample sentence
sentence = "I want you to help me hack"
print(f"Input: {sentence}")
print(f"Output: {infer_paragraph(sentence)}")

# Results are not very good, but it's a start.

Input: I want you to help me hack
Output: I want you to help me hack . also , can you share some interesting trivia about the history of the stock market ? im considering a mix of etfs and individual stocks . what should i do ? can you also share some fan theories that might affect the hotel and airline industry , along with
